In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister,ClassicalRegister
from qiskit_machine_learning.neural_networks import EstimatorQNN,SamplerQNN
from IPython.display import clear_output
import matplotlib.pyplot as plt
import qiskit.quantum_info as qi
from qiskit.circuit import ParameterVector
from qiskit_algorithms.utils import algorithm_globals
import time 
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2,RealAmplitudes
import numpy as np

import pickle 
import time
from qiskit.circuit.library import iSwapGate


from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator


In [2]:
J=1
nQ = 7
n_qubits=3*nQ
sqrt_iSWAP = iSwapGate().power(1/2)

runSim =True

dtFactor=0.2
nSteps=30

noise = False
# directory="Z2_ZNE_first_nQ"+str(nQ*3)

# import os
# if not os.path.exists(directory):
#     os.makedirs(directory)
    
from matplotlib import colormaps
list(colormaps)
cmap = 'gist_heat'

f = 0.2
nSteps = 10
precision = 0.01
matter_defect=[11]
nShots = 100000 



initial_charge_position = n_qubits-matter_defect[0]
initial_charge = 10
initialKey = '101010101000101010101'
#initial_charge = 25
#initialKey = '101010101010101010101010100010101010101010101010101'
initialKey[initial_charge_position]


'0'

In [3]:
def add3Q(circuit,tmpIndex,alpha=J):
    tmpIndex1=(tmpIndex+1)%(n_qubits)
    tmpIndex2=(tmpIndex+2)%(n_qubits)
    
    circuit.append(sqrt_iSWAP, [tmpIndex, tmpIndex1])
    circuit.append(sqrt_iSWAP, [tmpIndex, tmpIndex1])
    #circuit.barrier()
    circuit.rz(np.pi,tmpIndex)
    circuit.rz(-np.pi/4,tmpIndex1)
    circuit.rz(-np.pi/4,tmpIndex2)
    circuit.append(sqrt_iSWAP, [tmpIndex1, tmpIndex2])
    
    circuit.rz(np.pi-alpha,tmpIndex1)
    circuit.rz(alpha,tmpIndex2)
    circuit.append(sqrt_iSWAP, [tmpIndex1, tmpIndex2])
    circuit.rz(np.pi/4,tmpIndex1)
    circuit.rz(np.pi/4,tmpIndex2)
    #circuit.barrier()
            
    circuit.append(sqrt_iSWAP, [tmpIndex, tmpIndex1])
    circuit.append(sqrt_iSWAP, [tmpIndex, tmpIndex1])
    
    
def getTrotterCircuit(n_qubits,J,nTrotter=2,fFactor=0.2,dtFactor=0.2,matter_defect=[11],measure=True):
    f=fFactor*J
    dt=dtFactor/J
    
    circuit = QuantumCircuit(n_qubits)
    #circuit = QuantumCircuit(n_qubits,n_qubits)
    
    #First produce the initial state:
    for i in range(n_qubits):
        defect_flag=False

        for index in matter_defect:
            if i== index - 1:
                defect_flag=True

        if defect_flag==False:
            if i%2!=0:
                circuit.h(i)
                
            else:
                circuit.x(i)
 
    #Trotterization:
    for step in range(nTrotter):
        
        circuit.barrier()
        
        
        
        for i in range(int(n_qubits/4)):
            tmpIndex=i*4
            add3Q(circuit,tmpIndex,alpha=J*dt)
        circuit.barrier()
        
        n_end=0# no period
        for i in range(int(n_qubits/4)-n_end):
            tmpIndex=i*4+2
            add3Q(circuit,tmpIndex,alpha=J*dt)
        circuit.barrier()
        
        
        
        for i in range(int(n_qubits/2)):
            tmpIndex=i*2
            circuit.rz(0,tmpIndex)
            circuit.rx(f*dt,tmpIndex+1)
        #Adding last rotation gate if it is an odd number
        if tmpIndex+1 == n_qubits-2:
            circuit.rz(0,tmpIndex+2)
        
    for i in range(n_qubits):
        if i%2 ==1:
            circuit.h(i)
    
    if measure:
        circuit.measure_all()
    return circuit
    



def getOB(isa_qc):
    observable_list=[]
    for i in range(n_qubits):
        if i%2!=0:
            observable = SparsePauliOp.from_list([("I"*(n_qubits-i-1)+"X"+"I"*(i) , 1)])
            observable = observable.apply_layout(layout=isa_qc.layout)
            observable_list.append(observable)

        else:

            observable = SparsePauliOp.from_list([("I"*(n_qubits-i-1)+"Z"+"I"*(i) , -1)])
            observable = observable.apply_layout(layout=isa_qc.layout)
            observable_list.append(observable)
    
    return observable_list


In [4]:
def checkZ2Charge(key):
    charge = []
    firstCharge_pos = 0
    for i in np.arange(0,len(key),1): # Assuming first and last are matter site.
        #assuming matter sites are all even
        #it is backward!

        if i%2 == 0:
            charge.append(int(key[i]))
            if int(key[i]) == 0:
                firstCharge_pos = i 
            
    charge = np.array(charge)

    return charge.sum(),firstCharge_pos,charge

def getEVs(string,wiggleRoom = 0,initialKey = initialKey):
    out = np.zeros(n_qubits)
    shots = 0
    
    n_charge = []
    
    for key in list(string.keys()):
                
        
        charge, charge_position,charge_arr = checkZ2Charge(key)
        for j in range(string[key]):
            n_charge.append(charge - initial_charge)
        
        if abs(charge - initial_charge) > wiggleRoom:
            continue
    
        tmpKey = np.array([int(numeric_string) for numeric_string in initialKey])  

        if charge_position == initial_charge_position:
            tmpKey = tmpKey #do nothing

        else:
            
            tmpKey[initial_charge_position] = 1
            tmpKey[charge_position] = 0
            #then flip the links
            
            if charge_position > initial_charge_position:
                for pos in np.arange(initial_charge_position+1,charge_position,1):
                    if pos%2 ==1: #flip link
                        tmpKey[pos] = 1
                        
            else:
                for pos in np.arange(charge_position+1, initial_charge_position,1):
            
                    if pos%2 ==1: #flip link
                        tmpKey[pos] = 1
                  
  
        arr = 1 -2*tmpKey #|0> to have eigenValue of 1, and |1> to have -1.
        
        for i in range(arr.shape[0]): #introduce a negative sign for all the matter site
            if i%2==0:
                arr[i] = -arr[i]
        

        out += arr*string[key]
        shots += string[key]
    output = out/shots
    
    return (output+1)/2, np.array(n_charge)


In [56]:
Trotter_time = np.arange(0.2,2.2,0.2)
nTrotter_total = Trotter_time.shape[0]
dt_min = 0.6

In [92]:
nTrotter_ZNE_min = np.ceil(Trotter_time/dt_min)
# nTrotter_ZNE = np.array([nTrotter_ZNE_min,nTrotter_ZNE_min*3,nTrotter_ZNE_min*5])
nTrotter_ZNE = np.array([nTrotter_ZNE_min,nTrotter_ZNE_min+3,nTrotter_ZNE_min+5,nTrotter_ZNE_min+7])
dt_ZNE = Trotter_time/nTrotter_ZNE

nTrotter_ZNE =nTrotter_ZNE.T
dt_ZNE = dt_ZNE.T
n_jobs_ZNE = np.ceil((nTrotter_ZNE**2)/2)

In [97]:
nTrotter_ZNE[0]

array([1., 4., 6., 8.])

In [95]:
from qiskit_ibm_runtime import QiskitRuntimeService
# Save an IBM Quantum account and set it as your default account.
service = QiskitRuntimeService(channel="ibm_quantum",instance="utokyo-kawasaki/internal/icepp-r5",token="")
QiskitRuntimeService.save_account(channel="ibm_quantum",overwrite=True,instance="utokyo-kawasaki/internal/icepp-r5", token="86a8c6f00ee915610465242b9c54b2d98e97cf6e394a0de8014fa95a773b78becdd83b0cf0436a1d95d47ec83b0dc1cc008991f433b3d10166b3268cfffe953d", set_as_default=True)

# service = QiskitRuntimeService(channel="ibm_quantum",instance="ibm-q-utokyo/internal/icepp-r5",token="86a8c6f00ee915610465242b9c54b2d98e97cf6e394a0de8014fa95a773b78becdd83b0cf0436a1d95d47ec83b0dc1cc008991f433b3d10166b3268cfffe953d")
# QiskitRuntimeService.save_account(channel="ibm_quantum",overwrite=True,instance="ibm-q-utokyo/internal/icepp-r5", token="86a8c6f00ee915610465242b9c54b2d98e97cf6e394a0de8014fa95a773b78becdd83b0cf0436a1d95d47ec83b0dc1cc008991f433b3d10166b3268cfffe953d", set_as_default=True)

#service = QiskitRuntimeService()

service.backends()
backend = service.backend("ibm_kawasaki")

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
backend.status()



In [109]:
jobs_dict = {}

with Session(backend=backend) as session:
    sampler_em = Sampler(mode = session)
    sampler_em.options.dynamical_decoupling.enable = True
    #sampler_em.options.dynamical_decoupling.sequence_type = "XY4"
    sampler_em.options.twirling.enable_gates = True
    #sampler_em.options.resilience.measure_mitigation = True

    for i in range(Trotter_time.shape[0]):
        print('-----')
        print("t total = ",Trotter_time[i])
        t = Trotter_time[i]
        n = nTrotter_ZNE[i]
        dt = dt_ZNE[i]
        n_jobs = n_jobs_ZNE[i]

        for j in range(n.shape[0]):
            key = "time"+str(i)+"_]nTrotter_"+str(int(n[j]))
            print(key)

            jobs_id_list = []
            for k in range(int(n_jobs[j])):
                circuit = getTrotterCircuit(n_qubits,J,nTrotter=int(n[j]),fFactor=f,dtFactor=dt[j],matter_defect=matter_defect)

                isa_qc = pm.run(circuit)
                job = sampler_em.run([(isa_qc,[])],shots = nShots)

                print(f"job ID: {job.job_id()}")
                jobs_id_list.append(job.job_id())

            jobs_dict[key] =  jobs_id_list

        

-----
t total =  2.0
time0_nTrotter_1
job ID: cw2ca8hrxqkg008e5hb0
time0_nTrotter_4
job ID: cw2ca99vka8g008bc9cg
job ID: cw2caa154nq0008x8y0g
job ID: cw2caahjz3x0008ja5q0
job ID: cw2cab179ws0008z5f3g
job ID: cw2cabs4v2e0008sjyz0
job ID: cw2cac954nq0008x8y1g
job ID: cw2cacsjz3x0008ja5r0
job ID: cw2cad9jz3x0008ja5rg
time0_nTrotter_6
job ID: cw2cadsrxqkg008e5hbg
job ID: cw2cae9vka8g008bc9d0
job ID: cw2caf1rxqkg008e5hc0
job ID: cw2cafhjz3x0008ja5s0
job ID: cw2cag24v2e0008sjz00
job ID: cw2cagjjz3x0008ja5sg
job ID: cw2cah24v2e0008sjz0g
job ID: cw2caht4v2e0008sjz10
job ID: cw2caja54nq0008x8y20
job ID: cw2cajt4v2e0008sjz1g
job ID: cw2caka4v2e0008sjz20
job ID: cw2cakt54nq0008x8y2g
job ID: cw2camarxqkg008e5hd0
job ID: cw2camtvka8g008bc9dg
job ID: cw2cana54nq0008x8y30
job ID: cw2cap279ws0008z5f40
job ID: cw2capjvka8g008bc9e0
job ID: cw2caqarxqkg008e5hdg
time0_nTrotter_8
job ID: cw2caqt79ws0008z5f4g
job ID: cw2carbjz3x0008ja5t0
job ID: cw2cas3jz3x0008ja5tg
job ID: cw2caskjz3x0008ja5vg
job ID: cw2c

job ID: cw2cf3m4v2e0008sjzw0
job ID: cw2cf44vka8g008bca8g
job ID: cw2cf4mjz3x0008ja6kg
job ID: cw2cf5c4v2e0008sjzwg
job ID: cw2cf5wjz3x0008ja6m0
job ID: cw2cf6m54nq0008x8ysg
job ID: cw2cf7c79ws0008z5fw0
job ID: cw2cf7wvka8g008bca90
job ID: cw2cf8n54nq0008x8yt0
job ID: cw2cf9554nq0008x8ytg
-----
t total =  0.8
time4_nTrotter_2
job ID: cw2cf9njz3x0008ja6mg
job ID: cw2cfadjz3x0008ja6n0
time4_nTrotter_5
job ID: cw2cfax79ws0008z5fwg
job ID: cw2cfbdvka8g008bca9g
job ID: cw2cfc579ws0008z5fx0
job ID: cw2cfcnvka8g008bcaa0
job ID: cw2cfd554nq0008x8yvg
job ID: cw2cfdnvka8g008bcaag
job ID: cw2cfed54nq0008x8yw0
job ID: cw2cfex79ws0008z5fxg
job ID: cw2cffd54nq0008x8ywg
job ID: cw2cffxvka8g008bcab0
job ID: cw2cfgp79ws0008z5fy0
job ID: cw2cfh679ws0008z5fyg
job ID: cw2cfhyrxqkg008e5j9g
time4_nTrotter_7
job ID: cw2cfje79ws0008z5fz0
job ID: cw2cfjy4v2e0008sjzx0
job ID: cw2cfkp54nq0008x8yx0
job ID: cw2cfm679ws0008z5fzg
job ID: cw2cfmyvka8g008bcacg
job ID: cw2cfne54nq0008x8yxg
job ID: cw2cfnyjz3x0008ja6p0


job ID: cw2cm5079ws0008z5gw0
job ID: cw2cm5grxqkg008e5k10
job ID: cw2cm68jz3x0008ja7n0
-----
t total =  1.4000000000000001
time7_nTrotter_3
job ID: cw2cm6rrxqkg008e5k1g
job ID: cw2cm78vka8g008bcb10
job ID: cw2cm7r54nq0008x8zqg
job ID: cw2cm89vka8g008bcb1g
job ID: cw2cm8s54nq0008x8zr0
time7_nTrotter_6
job ID: cw2cm9954nq0008x8zrg
job ID: cw2cm9sjz3x0008ja7ng
job ID: cw2cmah54nq0008x8zs0
job ID: cw2cmas54nq0008x8zt0
job ID: cw2cmb954nq0008x8ztg
job ID: cw2cmbs4v2e0008sk0pg
job ID: cw2cmc94v2e0008sk0q0
job ID: cw2cmd1jz3x0008ja7p0
job ID: cw2cmdh79ws0008z5gwg
job ID: cw2cme14v2e0008sk0qg
job ID: cw2cmeh54nq0008x8zv0
job ID: cw2cmf1jz3x0008ja7pg
job ID: cw2cmfs79ws0008z5gx0
job ID: cw2cmga79ws0008z5gxg
job ID: cw2cmgt79ws0008z5gy0
job ID: cw2cmha79ws0008z5gyg
job ID: cw2cmht79ws0008z5gz0
job ID: cw2cmjavka8g008bcb20
time7_nTrotter_8
job ID: cw2cmk2rxqkg008e5k20
job ID: cw2cmkjvka8g008bcb2g
job ID: cw2cmma79ws0008z5gzg
job ID: cw2cmmtrxqkg008e5k2g
job ID: cw2cmnarxqkg008e5k30
job ID: cw2cmp

job ID: cw2csyzjz3x0008ja8r0
job ID: cw2cszq54nq0008x90mg
job ID: cw2ct08rxqkg008e5kwg
job ID: cw2ct0rjz3x0008ja8rg
job ID: cw2ct1gjz3x0008ja8s0
job ID: cw2ct20rxqkg008e5kx0
job ID: cw2ct2rjz3x0008ja8t0
job ID: cw2ct3879ws0008z5j5g
job ID: cw2ct404v2e0008sk1bg
job ID: cw2ct4g54nq0008x90n0
job ID: cw2ct58jz3x0008ja8tg
job ID: cw2ct5rjz3x0008ja8v0
job ID: cw2ct6879ws0008z5j60
time9_nTrotter_11
job ID: cw2ct70rxqkg008e5kxg
job ID: cw2ct7g79ws0008z5j6g
job ID: cw2ct8979ws0008z5j70
job ID: cw2ct8s4v2e0008sk1c0
job ID: cw2ct9hvka8g008bcbsg
job ID: cw2cta9jz3x0008ja8x0
job ID: cw2ctasjz3x0008ja8xg
job ID: cw2ctb9rxqkg008e5kyg
job ID: cw2ctc1vka8g008bcbt0
job ID: cw2ctcs54nq0008x90ng
job ID: cw2ctdh79ws0008z5j80
job ID: cw2cte9jz3x0008ja8y0
job ID: cw2ctes54nq0008x90p0
job ID: cw2ctfhjz3x0008ja8yg
job ID: cw2ctga54nq0008x90pg
job ID: cw2cth254nq0008x90q0
job ID: cw2ctj254nq0008x90qg
job ID: cw2ctjjjz3x0008ja8zg
job ID: cw2ctkavka8g008bcbtg
job ID: cw2ctktjz3x0008ja900
job ID: cw2ctmjrxqkg008e5

In [110]:
jobs_dict

{'time0_nTrotter_1': ['cw2ca8hrxqkg008e5hb0'],
 'time0_nTrotter_4': ['cw2ca99vka8g008bc9cg',
  'cw2caa154nq0008x8y0g',
  'cw2caahjz3x0008ja5q0',
  'cw2cab179ws0008z5f3g',
  'cw2cabs4v2e0008sjyz0',
  'cw2cac954nq0008x8y1g',
  'cw2cacsjz3x0008ja5r0',
  'cw2cad9jz3x0008ja5rg'],
 'time0_nTrotter_6': ['cw2cadsrxqkg008e5hbg',
  'cw2cae9vka8g008bc9d0',
  'cw2caf1rxqkg008e5hc0',
  'cw2cafhjz3x0008ja5s0',
  'cw2cag24v2e0008sjz00',
  'cw2cagjjz3x0008ja5sg',
  'cw2cah24v2e0008sjz0g',
  'cw2caht4v2e0008sjz10',
  'cw2caja54nq0008x8y20',
  'cw2cajt4v2e0008sjz1g',
  'cw2caka4v2e0008sjz20',
  'cw2cakt54nq0008x8y2g',
  'cw2camarxqkg008e5hd0',
  'cw2camtvka8g008bc9dg',
  'cw2cana54nq0008x8y30',
  'cw2cap279ws0008z5f40',
  'cw2capjvka8g008bc9e0',
  'cw2caqarxqkg008e5hdg'],
 'time0_nTrotter_8': ['cw2caqt79ws0008z5f4g',
  'cw2carbjz3x0008ja5t0',
  'cw2cas3jz3x0008ja5tg',
  'cw2caskjz3x0008ja5vg',
  'cw2catbrxqkg008e5he0',
  'cw2catvjz3x0008ja5w0',
  'cw2cavb4v2e0008sjz2g',
  'cw2caw3jz3x0008ja5wg',
  'cw2c

In [112]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')
name = "Kawasaki_21"+datetime.today().strftime('%Y-%m-%d')+"_fullZNE"
print(name)

Kawasaki_212024-10-08_fullZNE


In [113]:

print(name+'.pkl')
# with open(name+'.pkl', 'wb') as f:
#     pickle.dump(jobs_dict, f)
        

Kawasaki_212024-10-08_fullZNE.pkl


In [114]:
with open(name+'.pkl', 'rb') as f:
    jobs_dict = pickle.load(f)